Se realiza la carga de las librerías que serán necesarias para el analisís

In [ ]:
import openai
import os
from openai import AzureOpenAI
import pandas as pd
import os
import requests
import base64

Se procede a cargar el prompt que sera utilizando en los reviews de cada restaurante

In [44]:
with open("prompt.txt", "r", encoding="utf-8") as file:
    prompt = file.read()

ahora realizamos la carga de la información que contiene las reviews de los clientes

In [ ]:
file_id = '1PkwkxO77LTlUGimG3OstG3VqCNlXnYan'  
url = f'https://drive.google.com/uc?id={file_id}'
df_testing = pd.read_csv(url, encoding='utf-8')
df_testing

,status,nombre,direccion,latitud,longitud,place_id,calificacion,costo,tipos,total_ratings,url,comentario,estrellas,fecha_extraccion,fecha_publicacion,sentimiento
0,OPERATIONAL,El Corral Javeriana,NaN,4.628657,-74.065467,ChIJlTK-eCiaP44R_EBAP6zP_iM,4.4,2.0,"['restaurant', 'point_of_interest', 'food', 'e...",557.0,https://www.google.com/maps/place/?q=place_id:...,Comida: 5 Servicio: 5 Ambiente: 5,5.0,2024-09-25 00:00:00,2024-09-23 00:00:00,Positivo
1,OPERATIONAL,El Corral Javeriana,NaN,4.628657,-74.065467,ChIJlTK-eCiaP44R_EBAP6zP_iM,4.4,2.0,"['restaurant', 'point_of_interest', 'food', 'e...",557.0,https://www.google.com/maps/place/?q=place_id:...,Servicio: 3,3.0,2024-09-25 00:00:00,Hace 3 semanas,Neutral
2,OPERATIONAL,El Corral Javeriana,NaN,4.628657,-74.065467,ChIJlTK-eCiaP44R_EBAP6zP_iM,4.4,2.0,"['restaurant', 'point_of_interest', 'food', 'e...",557.0,https://www.google.com/maps/place/?q=place_id:...,Entregan super rápido Comida: 5 Servicio: 5 Am...,5.0,2024-09-25 00:00:00,Hace un mes,Positivo
3,OPERATIONAL,El Corral Javeriana,NaN,4.628657,-74.065467,ChIJlTK-eCiaP44R_EBAP6zP_iM,4.4,2.0,"['restaurant', 'point_of_interest', 'food', 'e...",557.0,https://www.google.com/maps/place/?q=place_id:...,Servicio Comí allí Tipo de comida Cena Precio ...,4.0,2024-09-25 00:00:00,Hace un mes,Positivo
4,OPERATIONAL,El Corral Javeriana,NaN,4.628657,-74.065467,ChIJlTK-eCiaP44R_EBAP6zP_iM,4.4,2.0,"['restaurant', 'point_of_interest', 'food', 'e...",557.0,https://www.google.com/maps/place/?q=place_id:...,La comida genial pero el local requiere más ai...,4.0,2024-09-25 00:00:00,Hace 2 meses,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9091,OPERATIONAL,The Chili Burger,NaN,4.633733,-74.073621,ChIJa8QQ9KebP44Rkac5NKXZdAw,4.5,NaN,"['restaurant', 'food', 'point_of_interest', 'e...",88.0,https://www.google.com/maps/place/?q=place_id:...,"Excelentes hamburguesas, sabores únicos y deli...",5.0,2024-04-10 00:00:00,Hace un año,Positivo
9092,OPERATIONAL,The Chili Burger,NaN,4.633733,-74.073621,ChIJa8QQ9KebP44Rkac5NKXZdAw,4.5,NaN,"['restaurant', 'food', 'point_of_interest', 'e...",88.0,https://www.google.com/maps/place/?q=place_id:...,Servicio Consumo en el lugar Tipo de comida Al...,5.0,2024-04-10 00:00:00,Hace un año,Positivo
9093,OPERATIONAL,The Chili Burger,NaN,4.633733,-74.073621,ChIJa8QQ9KebP44Rkac5NKXZdAw,4.5,NaN,"['restaurant', 'food', 'point_of_interest', 'e...",88.0,https://www.google.com/maps/place/?q=place_id:...,La comida rica y un bien servicio Comida: 4 Se...,4.0,2024-04-10 00:00:00,Hace un año,Positivo
9094,OPERATIONAL,The Chili Burger,NaN,4.633733,-74.073621,ChIJa8QQ9KebP44Rkac5NKXZdAw,4.5,NaN,"['restaurant', 'food', 'point_of_interest', 'e...",88.0,https://www.google.com/maps/place/?q=place_id:...,Un restaurante perfecto si les gusta lo picant...,5.0,2024-04-10 00:00:00,Hace un año,Positivo


Se crea un array que contiene el id unico de cada restauraten, observando que se tienen 176 restaurantes en total

In [31]:
restaurante_id=df_testing.place_id.unique()
print ( len(restaurante_id) )
restaurante_id

176


array(['ChIJlTK-eCiaP44R_EBAP6zP_iM', 'ChIJkQCpfiiaP44RzKgT6RU1c7w',
       'ChIJsd76eyiaP44RRmmxysDBH9s', 'ChIJd93M1yeaP44RUhHR-fWfa40',
       'ChIJa1Lm7TqbP44R6tGc9ikbMEE', 'ChIJGfaAGiaaP44RyfpYJY-0jfs',
       'ChIJ4YrFKyaaP44RqZYyxENMbwg', 'ChIJD8JmfCiaP44RsN-a8pdJ_uk',
       'ChIJe6pk7jmbP44RY8MkWeCTI4s', 'ChIJD6l7KM6bP44RkUQS4t4W4II',
       'ChIJzwmuhyiaP44RKifCe9jrBIk', 'ChIJf72OFLebP44RNzs8OxmDhUs',
       'ChIJDyi4AqKbP44RAnG2IjKmxy4', 'ChIJf0o7YSiaP44RpvjsyRs40ns',
       'ChIJ404DsembP44Rue6RqPWl-TE', 'ChIJDy2oAyiaP44RxNDXjfJacc4',
       'ChIJQ9xEAoiZP44R__CN9PoAM2c', 'ChIJ-z5Vpl-aP44RUf8AHmUGik0',
       'ChIJt-jH2tabP44Ra03yDRbhkOo', 'ChIJE1Jk94eZP44RtKRA7LHO2P0',
       'ChIJO6Yi8yiaP44RSOCZSGj7Ous', 'ChIJJUIsXCiaP44RZJv-p4eobDc',
       'ChIJO7yhXiiaP44RXvdA7xjse7w', 'ChIJy7u4lCiaP44RFC6fm8fP-k4',
       'ChIJ3-XLYyiaP44REkcTPhmXh_0', 'ChIJhYKXYCiaP44RhRXl9F9mSIA',
       'ChIJuwICiSiaP44RuZuxgOCA_2E', 'ChIJD8Y5XiiaP44RbqqNGWOr6no',
       'ChIJRfxVqNGbP44RRX4Ks3nP4E

Se realiza la creación de los parámetros que serán utilizados en el llamado del LLM Chat GPT 4o-mini

In [ ]:
# Configuration
API_KEY = #colocar api

headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

ENDPOINT = "https://gpt4ominiseb.openai.azure.com/openai/deployments/gpt-4o-mini-2/chat/completions?api-version=2024-02-15-preview"

ahora se genera la función que va a permitir realizar el llamado a chat gpt iterando sobre cada uno de los registros

In [70]:
df_resultados = pd.DataFrame(columns=["place_id", "Bueno", "Malo"])


In [ ]:
for restaurante in restaurante_id:

    #Filtro de información
    df_restaurante= df_testing[ df_testing["place_id"]==restaurante  ]
    #concatena resultados
    comments_concatenated = "\n".join([f"User {i + 1}: {row['comentario']}" for i, row in df_restaurante.iterrows()])

    #Creación del payload
    payload = {
        "messages": [
            {
            "role": "system",
            "content": [ {"type": "text",  "text": prompt } ]
            },
            {
            "role": "user",
            "content": [ {"type": "text",  "text": comments_concatenated } ] }
        ],
        "temperature": 0,
        "top_p": 0.95,
        "max_tokens": 1000
        }
    

    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    respuesta=response.json()['choices'][0]['message']['content']

    respuesta_pos= respuesta.split("**A tener en cuenta**:")[0].replace("**Lo que más les gusta**:", "").strip()
    respuesta_neg= respuesta.split("**A tener en cuenta**:")[1].strip()

    temp_df = pd.DataFrame({
        "place_id": [restaurante],  # Usamos idx como ejemplo de ID
        "Bueno": [respuesta_pos],
        "Malo": [respuesta_neg]
    })

    df_resultados = pd.concat([df_resultados, temp_df], ignore_index=True)

In [ ]:
#df_resultados.to_csv("resumen_restaurantes.csv")

In [ ]:
df_general=df_testing[["nombre","latitud", "longitud","place_id","calificacion","url"]]
df_general=df_general.drop_duplicates()

df_union=pd.merge(df_resultados, df_general,  on="place_id", how="left")
df_union = df_union.rename(columns={"latitud": "latitude", "longitud": "longitude","calificacion":"Calificacion","nombre":"Restaurante","url":"Reviews"})
df_union


,place_id,Bueno,Malo,Restaurante,latitude,longitude,Calificacion,Reviews
0,ChIJlTK-eCiaP44R_EBAP6zP_iM,La mayoría de los comensales elogia la calidad...,Existen comentarios sobre inconsistencias en e...,El Corral Javeriana,4.628657,-74.065467,4.4,https://www.google.com/maps/place/?q=place_id:...
1,ChIJkQCpfiiaP44RzKgT6RU1c7w,Los clientes destacan la excelente relación ca...,Algunos comensales han tenido experiencias neg...,Zarzamora,4.628818,-74.065628,4.4,https://www.google.com/maps/place/?q=place_id:...
2,ChIJsd76eyiaP44RRmmxysDBH9s,Muchos clientes destacan el sabor de las hambu...,Existen quejas sobre la inconsistencia en la c...,Presto Javeriana,4.628697,-74.065697,4.0,https://www.google.com/maps/place/?q=place_id:...
3,ChIJd93M1yeaP44RUhHR-fWfa40,Los clientes elogian la calidad y el sabor de ...,Algunos comensales han expresado preocupacione...,El Carnal - Javeriana,4.628217,-74.065064,4.4,https://www.google.com/maps/place/?q=place_id:...
4,ChIJa1Lm7TqbP44R6tGc9ikbMEE,Los clientes destacan la calidad y el sabor de...,Algunos comensales han mencionado que la comid...,El Mero Burrito,4.629138,-74.065123,4.7,https://www.google.com/maps/place/?q=place_id:...
...,...,...,...,...,...,...,...,...
171,ChIJvViR6nWbP44RCU4A1k_nTMA,Los clientes elogian la calidad y el sabor de ...,Algunas reseñas indican que la calidad de cier...,El Garnachan,4.633389,-74.072967,4.6,https://www.google.com/maps/place/?q=place_id:...
172,ChIJrfZZRyyaP44RC4JTBHR9yrY,Muchos comensales destacan la calidad y el sab...,Existen quejas sobre la inconsistencia en la c...,Restaurante A Comer Se Dijo,4.633295,-74.072630,4.2,https://www.google.com/maps/place/?q=place_id:...
173,ChIJnbDOOyyaP44RjYXDXxTkVsQ,Los clientes disfrutan mucho del ambiente del ...,Algunos comensales han mencionado que la calid...,CASA MANIGUA,4.633470,-74.072375,4.7,https://www.google.com/maps/place/?q=place_id:...
174,ChIJHRZGXASbP44Re3-DZ_8Do-g,Los clientes disfrutan del ambiente acogedor y...,Existen quejas sobre la inconsistencia en el s...,Soul 45,4.633500,-74.072321,4.6,https://www.google.com/maps/place/?q=place_id:...


In [87]:
df_union.to_csv("resumen_streamlit.csv", sep=";", index=False)